# Données géospatiales : dataviz avec plotly en mode BYOD 

1. **Introduction**
2. **Plotly**
3. **treemap et sunburst**
4. **Cartes choroplèthes**

## 1. En guise d'introduction

La richesse des librairies graphiques Python...

In [ ]:
from IPython import display

# source Jake Vanderplas PyCon 2017
# voir https://sophiamyang.medium.com/python-visualization-landscape-3b95ede3d030

display.Image("./data/landscape-colors.png")

In [ ]:
# https://pyviz.org

display.IFrame("https://pyviz.org/tools.html#geospatial", width=800, height=400)

Pour les cartographies :
- **basemap** : The matplotlib basemap toolkit is a library for plotting 2D data on maps in Python.
- **cartopy** : Cartopy is a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses.
- **ipleaflet** : Interactive maps in the Jupyter notebook
- **geopandas** : GeoPandas is an open source project to make working with geospatial data in python easier.
- **geoviews** : GeoViews is a Python library that makes it easy to explore and visualize any data that includes geographic locations. 
- **plotly** : Cf. ci-dessous...

## 2. Plotly pour des représentations interactives

Plotly est basé sur javascript. Il permet de créer des graphiques interactifs. Il est édité par la société canadienne *Plotly* et est également intégré dans le package **dash** de création de dashboards.

On va utiliser le module **plotly.express** dont l'API est particulièrement simple. Il existe un module plus complet : **plotly.graph_objects**.

In [ ]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px

## 3. treemap et sunburst

Une treemap (resp. sunburst) ou carte proportionnelle est une représentation rectangulaire (resp. circulaire) de données hiérarchiques dans un espace limité.

In [ ]:
# dataset
fortune = (pd.read_csv("./data/Fortune_1000.csv", na_values="-")
           .fillna(0)
          )

fortune.head()

In [ ]:
# treemap
px.treemap(fortune,
          path=["sector", "company"],
          values="Market Cap")

In [ ]:
# on limite aux 4 première capitalisations par secteur
fortune2 = (fortune.groupby("sector")
            .apply(lambda g: g.nlargest(4, "Market Cap"))
           )

# sunburst
px.sunburst(fortune2,
          path=["sector", "company"],
          values="Market Cap")

## 4. Cartes choroplèthes

Une carte *choroplèthe* (du grec χῶρος  : « zone/région » et πλῆθος  : « grand nombre, multitude ») est une carte thématique où les régions sont colorées ou remplies d'un motif qui montre une mesure statistique.

Pour produire une carte choroplèthe, il faut récupérer un fichier *geojson* de la partie du monde considérée. Ce fichier contient entre autres 2 informations importantes :
- Une clé qui désigne chaque sous-région,
- La description d'un polygone ou d'un multi-polygones sous la forme d'une liste de coordonnées (latitudes et longitudes),

Il faut ensuite faire correspondre les valeurs de la colonne du DataFrame qui contient les données relatives à chaque sous-région avec celles de la clé du fichier *geojson*.

Il est possible ensuite d'utiliser différents fonds de cartes et différents nuanciers.

Le fichier "./data/departements.geojson" utilisé ci-après provient du site : https://france-geojson.gregoiredavid.fr/ et il faut analyser le fichier pour trouver la clé correspondant à chaque sous-région.

In [ ]:
import json

with open("./data/departements.geojson") as f:
    departements = json.loads(f.read())

departements

Analyse du fichier geojson :
- departements["features"] est une liste de 96 dictionnaires (les différents départements avec '2A' et '2B')
- chaque dictionnaire contient les clés :
    - 'type': 'Feature'
    - 'geometry': informations géométriques (polygones...)
    - 'properties' : informations d'identification qui permettent de référencer chaque dictionnaire

In [ ]:
# analyse du dictionnaire "departements"
departements["features"][0]["properties"]

In [ ]:
# liste des valeurs de la clé "code"
codes = sorted([f["properties"]["code"] for f in departements["features"]])
print(*codes)

In [ ]:
# dataset 
geo = pd.read_csv("data/correspondance-code-insee-code-postal.csv",
                 sep=";",
                 )

geo.head()

On voit que l'on va pouvoir utiliser la colonne "Code Département" pour référencer les dictionnaires du fichier *geojson*. La clé d'accès sera : "properties.code".

In [ ]:
# liste des "Code Département"
codes2 = sorted(geo["Code Département"].unique())
print(*codes2)

On calcule le nombre d'habitants par département.

In [ ]:
# population des départements
df = (geo.groupby("Code Département", as_index=False)["Population"]
      .sum()
     )

df.head()

On produit la carte directement en donnant notamment les 4 informations :
- le dictionnaire issu du fichier *geojson* : `geojson=departements`,
- la clé dans le DataFrame : `locations='Code Département'`
- la clé dans le fichier *geojson* : `featureidkey='properties.code'`
- la grandeur représentée : `color='Population'`

In [ ]:
# choropleth_mapbox
px.choropleth_mapbox(data_frame=df,
                     geojson=departements,
                     locations='Code Département',
                     color='Population',
                     featureidkey='properties.code',
                     color_continuous_scale="teal",
                     mapbox_style="carto-positron",
                     zoom=4.0,
                     center = {"lat": 47.0, "lon": 0.0},
                     opacity=0.5,
                     labels={'Population': 'Population en milliers'}
                    )

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 1</h3>
    <ul>
        <li>Produire la même carte choroplèthe en utilisant non pas le n° mais le nom du département. Que constate-t-on ?</li>
    </ul>
</div>

In [ ]:
# %load exo_1.py

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 2</h3>
    <ul>
        <li>Produire une carte choroplèthe avec la moyenne de l'Altitude Moyenne de chaque département, en utilisant le fond de carte "stamen-terrain" et le nuancier "reds".</li>
    </ul>
</div>

In [ ]:
# %load exo_2.py

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 3 - pour ceux qui n'ont pas de dataset</h3>
    <ul>
        <li>Charger le fichier "./data/arrondissements.geojson" et analyser la clé d'accès aux arrondissements (source: https://github.com/fxjollois/donnees)</li>
        <li>Charger et préparer le fichier Airbnb "./data/airbnb.csv"</li>
        <li>Calculer le prix moyen des hébergements Airbnb par arrondissement parisien.</li>
        <li>Analyser les écarts entre les n° d'arrondissements avec la clé d'accès aux arrondissements dans le fichier <i>geojson</i> et opérer les transformations nécessaires.</li>
        <li>Produire une carte choroplèthe avec la moyenne du prix des hébergements Airbnb de chaque arrondissement parisien, en utilisant le fond de carte "open-street-map" et le nuancier "reds".</li>
    </ul>
</div>

In [ ]:
# %load exo_3.py

<div class="alert alert-info">
    <h3><i class="fa fa-info-circle"></i> Documentation</h3>
    <p><a href="https://plotly.com/python/plotly-express/">Plotly Express</a></p>
    <p><a href="https://plotly.com/python/builtin-colorscales/">Built-in Continuous Color Scales</a></p>
    <p><a href="https://plotly.com/python/mapbox-layers/">Mapbox Map Layers</a></p>
</div>